In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

sys.path.append('../..')

import libraries as lib

In [2]:
lib.customize_style("notebook")

plot_dict = {
    "energy": "Energy",
    "cos_zenith": "Cosine Zenith",
}


In [3]:
smearing_level = "50"


In [ ]:
if __name__ == "__main__":
    # Define the path to the ROOT file
    
    path = f"/sps/km3net/users/mchadoli/Swim/Data/events/ANTARES_Smeared_{smearing_level}/"
    root_file = f"antares_smeared_{smearing_level}.root"
    COLUMNS = [
        "energy_true",
        "cos_zenith_true",
        "energy_smeared",
        "cos_zenith_smeared", 
        ]

    # Load the ROOT file into a DataFrame
    df = lib.load_large_rootfile_to_df(os.path.join(path, root_file), columns=COLUMNS)

    # Rounding the energy and cos_zenith columns
    df["energy_true"] = df["energy_true"].round(3)
    df["cos_zenith_true"] = df["cos_zenith_true"].round(3)

In [5]:
def _plot_hist_fit(
        df,
        parameter = "energy",
        bins=100,
        value= 20,
        smearing_level = "50",
):
    
    
    print(f"Plotting histogram and Gaussian fit for {parameter} with true value {value}")
    smeared_column = f"{parameter}_smeared"
    true_column = f"{parameter}_true"
    data = df[df[true_column] == value]
    Alpha = len(data) 

    # 2. Fit a Gaussian distribution to the data
    mu, std = norm.fit(data[smeared_column])    

    # 3. Plot the histogram with counts (density=False automatically uses counts)
    fig, ax = plt.subplots()
    data.hist(
        column=smeared_column, 
        bins= bins, 
        figsize=(10, 6), 
        ax=ax, 
        histtype='step', 
        hatch = '///',
        lw=2, 
        label='Histogram')
    
    # 4. Create x values for the fitted Gaussian curve
    xmin, xmax = plt.xlim()  # Get x-axis limits from the histogram
    if (parameter == "cos_zenith") & (xmin < -1):
        xmin = -1
    if (parameter == "cos_zenith") & (xmax > 1):
        xmax = 1
        

    x_values = np.linspace(xmin, xmax, 1000)  # Create 1000 x values between xmin and xmax

    # 5. Plot the Gaussian curve directly, scaling to match histogram's y-axis (counts)
    p = norm.pdf(x_values, mu, std) * Alpha * (xmax - xmin) / bins

    ax.plot(
        x_values, 
        p,
        "--", 
        lw=2, 
        label=f'Fitted Gaussian\n $\\mu$={mu:.3f}, $\\sigma$={std:.3f}')
    
    # 6. Customize plot
    ax.set_xlabel(f'Smeared {plot_dict[parameter]} [GeV]')
    ax.set_ylabel('Counts')
    ax.set_title(f'True value {value} with smearing level {smearing_level} %')
    ax.legend(frameon=False)

    # 7. Show the plot
    fig.tight_layout()
    fig.savefig(f"../plots/test/histogram_{smearing_level}_{parameter}_{value}.png")


In [ ]:
for value in [0, 0.1, 0.01, 0.5, -0.5]:
    _plot_hist_fit(df, parameter="cos_zenith", value=value, smearing_level=smearing_level)

for value in [15, 50, 70]:
    _plot_hist_fit(df, parameter="energy", value=value, smearing_level=smearing_level, bins = 40)